In [2]:
#Se imprtan las librerias necesarias

import ast
import pandas as pd
import numpy as np

## Etapas propuestas para este proyecto

* cargar cada archivo, decomprimirlo, decodearlo

* eliminar keys duplicadas

* flatenizar jsons anidados

* eliminar filas vacias

* guardar como csv

# movies_dataset ETL

In [3]:
#lectura 
movies = pd.read_csv('data/movies_dataset.csv',delimiter = ',',encoding = "utf-8",low_memory=False)
movies.head(1)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [4]:
# se guarda el original
movies_ori = movies.copy()

In [20]:
#dimensiones
movies.shape

(45466, 24)

In [21]:
#se cuentan valores unicos de id e imdb_id
print(f"id distintos: {movies['id'].nunique()}")
print(f"id nulos: {movies['id'].isnull().sum()}")
print(f"imdb_id distintos: {movies['imdb_id'].nunique()}")
print(f"imdb_id nulos: {movies['imdb_id'].isnull().sum()}")

id distintos: 45436
id nulos: 0
imdb_id distintos: 45417
imdb_id nulos: 17


In [27]:
movies = movies_ori

In [5]:
#se eliminan duplicados y nulos de imdb_id
movies.drop_duplicates(subset="imdb_id", inplace=True)
print(f"cantidad de imdb_id sin duplicados: {movies['imdb_id'].size}")

#se completan con ceros los nulos de revenue, budget
movies['revenue'].fillna(0)
movies['budget'].fillna(0)

#se eliminan nulos

movies = movies.dropna(subset=['imdb_id','release_date'], how = 'any')

print(f"cantidad de imdb_id sin nulos: {movies['imdb_id'].size}")



cantidad de imdb_id sin duplicados: 45418
cantidad de imdb_id sin nulos: 45333


In [24]:
movies.shape

(45333, 24)

In [6]:
#eliminar columnas video,imdb_id,adult,original_title,poster_path y homepage
movies = movies.drop(columns=['video','imdb_id','adult','original_title','poster_path','homepage'])
#dimensiones
movies.shape

(45333, 18)

In [44]:
#hay una fecha no valida
movies['release_date'].str.len().unique()

array([10,  1])

In [7]:
#eliminar fila con fecha no valida
movies = movies.drop(movies[movies.release_date.str.len() == 1].index)
movies.shape


(45332, 18)

In [8]:
#formatear fecha
movies['release_date'] = pd.to_datetime(movies['release_date'], format='%Y-%m-%d')

In [9]:
#crear columna release_year a partir del año de relase_date
movies['release_year'] = movies['release_date'].dt.year.astype('Int64')

In [13]:
#crear campo return = revenue / budget
#df['col4'] = df['col2'].div(df['col3']).replace(np.inf,0).fillna(0)
movies['return'] = movies['revenue'].astype(int).div(movies['budget'].astype(int)).replace(np.inf,0).fillna(0)

In [14]:
# Considero que no es necesario desanidar ninguno de los campos factibles de desanidar
movies.info()



<class 'pandas.core.frame.DataFrame'>
Index: 45332 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4483 non-null   object        
 1   budget                 45332 non-null  object        
 2   genres                 45332 non-null  object        
 3   id                     45332 non-null  object        
 4   original_language      45321 non-null  object        
 5   overview               44393 non-null  object        
 6   popularity             45332 non-null  object        
 7   production_companies   45332 non-null  object        
 8   production_countries   45332 non-null  object        
 9   release_date           45332 non-null  datetime64[ns]
 10  revenue                45332 non-null  float64       
 11  runtime                45088 non-null  float64       
 12  spoken_languages       45332 non-null  object        
 13  status

In [26]:
#guardar como csv
movies.to_csv(r'data/movies_depurado.csv',index=False)

### Transformaciones exclusivas para ser consumidas por la api

In [94]:
#se crea una BD exclusiva para se consumida por la api
movies_depurado =  pd.read_csv('data/movies_depurado.csv', usecols=['id','release_date','title','vote_average','vote_count','popularity','return', 'budget', 'revenue'], parse_dates=["release_date"], date_format='%Y-%m-%d', delimiter = ',',encoding = "utf-8",low_memory=False)
movies_depurado.head()

,budget,id,popularity,release_date,revenue,title,vote_average,vote_count,return
0,30000000,862,21.946943,1995-10-30,373554033.0,Toy Story,7.7,5415.0,12.451801
1,65000000,8844,17.015539,1995-12-15,262797249.0,Jumanji,6.9,2413.0,4.043035
2,0,15602,11.712900,1995-12-22,0.0,Grumpier Old Men,6.5,92.0,0.000000
3,16000000,31357,3.859495,1995-12-22,81452156.0,Waiting to Exhale,6.1,34.0,5.090760
4,0,11862,8.387519,1995-02-10,76578911.0,Father of the Bride Part II,5.7,173.0,0.000000


In [35]:
movies_depurado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45332 entries, 0 to 45331
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   release_date  45332 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 354.3 KB


In [41]:
movies_depurado.head()

,release_date,mes,week_day
0,1995-10-30,10,0
1,1995-12-15,12,4
2,1995-12-22,12,4
3,1995-12-22,12,4
4,1995-02-10,2,4


In [95]:
#guardar como csv
movies_depurado.to_csv(r'data/movies_api.csv',index=False)

# credits ETL

Se utiliza ast por que el json no esta en un formato standard, las keys estan encerradas en comillas simples

In [3]:
#lectura y copia de seguridad
credits = pd.read_csv('data/credits.csv',delimiter = ',',encoding = "utf-8",low_memory=False)
credits_ori = credits.copy()
credits.head(1)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


In [28]:
#dimensiones
credits.shape

(25799, 3)

In [7]:
#preparo un archivo mas chico para pruebas
credits_small = credits.head(20)
#guardar como csv
credits_small.to_csv(r'data/credits_small.csv',index=False)

In [30]:
# se desanidará y se conservaran los siguientes campos:
# gender ,id (id_job) ,name ,job (para los actores será 'actor')

data = []
for indice, fila in credits.iterrows():
    mov_id = fila['id']
    cast = fila['cast']
    crew = fila['crew']

    actores = ast.literal_eval(cast)
    for actor in actores:
        ele = {'mov_id':mov_id,'gender':actor['gender'],'id_job':actor['id'],'name':actor['name'],'job':'actor'}
        data.append(ele)        
    
    colabs = ast.literal_eval(crew)
    for col in colabs:
        ele = {'mov_id':mov_id,'gender':col['gender'],'id_job':col['id'],'name':col['name'],'job':col['job']}
        data.append(ele)  
    
df_credits = pd.DataFrame(data)
#dimensiones
print(df_credits.shape)
df_credits.info()

df_credits.to_csv(r'data/credits_flat.csv',index=False)


(1026788, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1026788 entries, 0 to 1026787
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   mov_id  1026788 non-null  int64 
 1   gender  1026788 non-null  int64 
 2   id_job  1026788 non-null  int64 
 3   name    1026788 non-null  object
 4   job     1026788 non-null  object
dtypes: int64(3), object(2)
memory usage: 39.2+ MB
